In [1]:
import pandas as pd
import numpy as np

from scipy.stats import norm, gaussian_kde, percentileofscore

pd.options.display.max_columns = None
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats

# For Shot Chart
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import cm
from matplotlib.patches import Circle, Rectangle, Arc, ConnectionPatch
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, BoundaryNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch

/Users/teddyd/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/Users/teddyd/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from matplotlib.patches import Circle, Rectangle, Arc

def draw_court(ax=None, color='black', lw=2, outer_lines=False):
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -7.5), 60, -1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                         fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color="red")

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color="red")
    
    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    
################# Below this line is my new code #######################
    
    #5 feet behind 3 point line
    four_arc = Arc((0, 0), 575, 575, theta1=22, theta2=158, linewidth=lw, color='red')
    
    corner_three_zone_a = Rectangle((-250, 92.5), 30, 0, linewidth=lw, color="red")
    corner_three_zone_b = Rectangle((220, 92.5), 30, 0, linewidth=lw, color="red")
    #low_left_zone = Rectangle((-125, -47.5), 125, 190, linewidth=lw, color="red", fill=False)
    #low_right_zone = Rectangle((0, -47.5), 125, 190, linewidth=lw, color="red", fill=False)
    baseline_jump_left = Rectangle((-220,-47.5), 95, 140, linewidth=lw, color="red", fill=False)
    baseline_jump_right = Rectangle((125,-47.5), 95, 140, linewidth=lw, color="red", fill=False)
    
    # Drawing 3 lines that will make divisions in mid range & 3-pointer territories
    left_line = Rectangle((-125, 92.5), 0, 167, linewidth=lw, color="red")
    right_line = Rectangle((125, 92.5), 0, 167, linewidth=lw, color="red")
    middle_line = Rectangle((0, 40), 0, 247.5, linewidth=lw, color="red")
    
    #adding line at FT that connects zones
    ft_zone_line = Rectangle((-125, 142.5), 250, 0, linewidth=lw, color="red")
    
    #adding two lines that connect restricted zones
    left_restricted_line = Rectangle((-40, -47.5), 0, 47.5, linewidth=lw, color="red")
    right_restricted_line = Rectangle((40, -47.5), 0, 47.5, linewidth=lw, color="red")
    
################# Above this line is my new code #######################

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, inner_box, outer_box,top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc,center_outer_arc,
                      center_inner_arc, four_arc, corner_three_zone_a, corner_three_zone_b,
                      baseline_jump_left, baseline_jump_right,
                     left_line, right_line, middle_line, ft_zone_line,
                     left_restricted_line, right_restricted_line]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax

In [3]:
#creating dataset
def get_player_shotchartdetail(player_name, season_id):
    """
    Parameters
    ----------
    player_name: name of the player with Capital
    season_id: ex. 2012-13
    """
    
    # player dictionary
    nba_players = players.get_players()
    player_dict = [player for player in nba_players if player['full_name'] == player_name][0]
    
    # career df
    career = playercareerstats.PlayerCareerStats(player_id=player_dict['id'])
    career_df = career.get_data_frames()[0]
    
    # team id during the season
    team_id = career_df[career_df['SEASON_ID'] == season_id]['TEAM_ID']
    
    # shotchardtdetail endpoint
    shotchartlist = shotchartdetail.ShotChartDetail(team_id=int(team_id), 
                                                   player_id=int(player_dict['id']), 
                                                   season_type_all_star='Regular Season', 
                                                   season_nullable=season_id,
                                                   context_measure_simple="FGA").get_data_frames()
    
    return shotchartlist[0], shotchartlist[1]

In [4]:
shot_df, league_avg = get_player_shotchartdetail('Joel Embiid', '2020-21')

In [5]:
shot_df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022000013,7,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,11,43,Missed Shot,Fadeaway Jump Shot,2PT Field Goal,Mid-Range,Center(C),8-16 ft.,14,-4,147,1,0,20201223,PHI,WAS
1,Shot Chart Detail,0022000013,48,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,8,1,Made Shot,Alley Oop Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-2,21,1,1,20201223,PHI,WAS
2,Shot Chart Detail,0022000013,58,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,6,45,Missed Shot,Fadeaway Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,15,94,120,1,0,20201223,PHI,WAS
3,Shot Chart Detail,0022000013,83,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,4,56,Made Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,10,-103,34,1,1,20201223,PHI,WAS
4,Shot Chart Detail,0022000013,95,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,4,13,Made Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,20,9,200,1,1,20201223,PHI,WAS
